<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [19]:
from config import *

import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [20]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [21]:
device = get_default_device()

In [39]:
class trainDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        X = self.x[idx]
        y = self.y[idx]
        # print(X, y)
        return torch.LongTensor(X), torch.tensor(y)

In [23]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [24]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()

        self.sigmoid = nn.Sigmoid()
        
        self.feedforward_clf = nn.Sequential(
            nn.Linear(768, 400),
            nn.ReLU(),
            nn.Linear(400, 200),
            nn.ReLU(),
            nn.Linear(200, 400),
            nn.ReLU(),
            nn.Linear(400, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 1)
        )

    def forward(self, x):
        output = self.sigmoid(self.feedforward_clf(x))

        return output

In [44]:
def train(train_dl, model, criterion, optimizer, epoch):
    losses = AverageMeter()

    model.train()

    for i, (X, y) in enumerate(train_dl):
        y_hat = model(X.float()).squeeze(1)
        loss = criterion(y_hat.float(), y.float())

        losses.update(loss.data, X.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(' Epoch: {ep}\t'
          ' Loss {loss.avg:.5f}\t'.format(ep=epoch, loss=losses))

    return loss.cpu().data.numpy()


def test_metrics(test_dl, model):
    model.eval()
    TPs = 0
    FPs = 0
    FNs = 0
    losses = AverageMeter()
    accuracies = AverageMeter()
    
    x_list = []
    y_list = []
    for i, (X, y) in enumerate(train_dl):
        y_hat = model(X.float()).squeeze(1)

        loss_func = nn.BCELoss()
        loss = loss_func(y_hat.float(), y.float())

        clf_binary = y_hat.data.cpu().numpy() > 0.5
        correct = (clf_binary == y.data.cpu().numpy()).sum()
        accuracy = correct / clf_binary.shape[0]

        for i in range(len(y)):
            if y[i] == 1:
                if y_hat[i] > 0.5:
                    TPs += 1
                else:
                    FNs += 1
            elif y_hat[i] > 0.5:
                FPs += 1

        losses.update(loss.data, X.size(0))
        accuracies.update(accuracy, X.size(0))

    return losses, accuracies, TPs, FPs, FNs

In [26]:
with open(triplet_train_set_path, 'rb') as f:
    train_set = pickle.load(f)
# with open(triplet_test_set_path, 'rb') as f:
#     test_set = pickle.load(f)

with open(clf_test_set_path, 'rb') as f:
    clf_test_set = pickle.load(f)
    
with open(ingred_emb_path, 'rb') as f:
    ingred_emb = pickle.load(f)    

In [27]:
n = len(train_set)
train_recipe_cart_data = [[[e[0]+e[1], 1], [e[0]+e[2], 0]] for i, e in enumerate(train_set[:round(0.7*n)]) if i % 3 == 0]
# valid_recipe_cart_data = [[[e[0]+e[1], 1], [e[0]+e[2], 0]] for i, e in enumerate(train_set[round(0.7*n):]) if i % 3 == 0]
test_recipe_cart_data = [[[e[0]+e[1], 1], [e[0]+e[2], 0]] for i, e in enumerate(test_set) if i % 3 == 0]
# test_recipe_cart_data = clf_test_set

In [28]:
X_y_train =  [i for ll in train_recipe_cart_data for i in ll]
X_train = [i[0] for i in X_y_train]
y_train = [i[1] for i in X_y_train]

X_emb_train = []
for i in X_train:
    n = np.zeros(768)
    for j in i:
        n += ingred_emb[j]
    n = n/len(i)
    X_emb_train.append(n)

In [ ]:
test_recipe_cart_data[0]

[[2892, 10373], [9589, 2588], 0]

In [31]:
X_y_test =  [i for ll in test_recipe_cart_data for i in ll]
# X_y_test = test_recipe_cart_data
X_test = [i[0]+i[1] for i in X_y_test]
y_test = [i[2] for i in X_y_test]

X_emb_test = []
for i in X_test:
    n = np.zeros(768)
    for j in i:
        n += ingred_emb[j]
    n = n/len(i)
    X_emb_test.append(n)

In [32]:
model = NN()
model.to(device)

NN(
  (sigmoid): Sigmoid()
  (feedforward_clf): Sequential(
    (0): Linear(in_features=768, out_features=400, bias=True)
    (1): ReLU()
    (2): Linear(in_features=400, out_features=200, bias=True)
    (3): ReLU()
    (4): Linear(in_features=200, out_features=400, bias=True)
    (5): ReLU()
    (6): Linear(in_features=400, out_features=200, bias=True)
    (7): ReLU()
    (8): Linear(in_features=200, out_features=100, bias=True)
    (9): ReLU()
    (10): Linear(in_features=100, out_features=50, bias=True)
    (11): ReLU()
    (12): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [33]:
criterion = nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters())

In [34]:
train_ds = trainDataset(X_emb_train, y_train)
train_dl = DataLoader(train_ds, batch_size=2000, shuffle=True)
train_dl = DeviceDataLoader(train_dl, device)

test_ds = trainDataset(X_emb_test, y_test)
test_dl = DataLoader(test_ds, batch_size=2000, shuffle=True)
test_dl = DeviceDataLoader(test_dl, device)

In [35]:
#check
X_t,y_t = next(iter(train_dl))

X_t, y_t

(tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 tensor([1, 0, 1,  ..., 1, 0, 0]))

In [45]:
for epoch in range(10):
    print('=== Start training epoch {} ==='.format(epoch))
    loss = train(train_dl, model, criterion, optimizer, epoch)

    # val_loss, val_acc, val_prec, val_recl = test_metrics(test_dl, model)
    # print(' -> valid loss: {loss:.5f}\t'
    #       ' valid acc: {acc:.5f}\t'
    #       ' valid prec: {prec:.5f}\t'
    #       ' valid recl: {recl:.5f}\t'.format(loss=val_loss.avg, acc=val_acc.avg, prec=val_prec, recl=val_recl))
    
    val_loss, val_acc, v_TPs, v_FPs, v_FNs = test_metrics(test_dl, model)
    print(' -> valid loss: {loss:.5f}\t'
          ' valid acc: {acc:.5f}\t'
          ' valid prec: {prec:.5f}\t'
          ' valid recl: {recl:.5f}\t'.format(loss=val_loss.avg, acc=val_acc.avg, prec=v_TPs/(v_TPs+v_FPs), recl=v_TPs/(v_TPs+v_FNs)))


=== Start training epoch 0 ===
 Epoch: 0	 Loss 0.59069	
 -> valid loss: 0.58996	 valid acc: 0.68783	 valid prec: 0.68148	 valid recl: 0.70533	
=== Start training epoch 1 ===
 Epoch: 1	 Loss 0.59053	
 -> valid loss: 0.59018	 valid acc: 0.68783	 valid prec: 0.68147	 valid recl: 0.70537	
=== Start training epoch 2 ===


KeyboardInterrupt: 

In [1]:
(2*0.68562*0.69211)/(0.68562+0.69211)

0.6888497139497579

In [46]:
v_TPs

35499

In [ ]:
y_hat_test = clf.predict(X_emb_test)

In [ ]:
TP = sum((np.array(y_hat_test) == np.array(y_test))&(np.array(y_test) ==1))

In [ ]:
predPs = sum(np.array(y_hat_test) == 1)

In [ ]:
actualPs = sum(np.array(y_test) == 1)

In [ ]:
precision = TP/predPs
recall = TP/actualPs

In [ ]:
precision, recall